In [1]:
import os 
import numpy as np 
import pandas as pd 
from pandas import Series, DataFrame 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import OneHotEncoder 
from sklearn import svm
from sklearn.datasets import samples_generator
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVR
import matplotlib.pyplot as plt
import ssl 
import xgboost as xgb
from xgboost import XGBRFRegressor
from xgboost.sklearn import XGBRegressor
import pymysql
import os
from sklearn.externals import joblib

# used to fix Python SSL CERTIFICATE_VERIFY_FAILED
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None)):
    ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
project_data_path = '/Users/charles/Desktop/Project/LongTan_File/'
longtan_file_2018 = os.path.join(project_data_path, 'LongTan_complete_2018.csv')
print('Path of read in data: %s' % (longtan_file_2018))

Path of read in data: /Users/charles/Desktop/Project/LongTan_File/LongTan_complete_2018.csv


In [3]:
project_data = pd.read_csv(longtan_file_2018)

In [4]:
project_data.head()

,station,Year,Month,Day,Time,AMB_TEMP,CO,NO,NOx,O3,PM10,PM2.5,RAINFALL,RH,SO2,WD_HR,WIND_DIREC,WIND_SPEED,WS_HR
0,LT,2018,1,1,0,14.0,0.25,0.4,2.4,46.0,58.0,19.0,0.0,75.0,1.0,71.0,71.0,7.1,6.7
1,LT,2018,1,1,1,14.1,0.25,0.8,1.7,49.0,58.0,23.0,0.0,75.0,1.4,69.0,72.0,6.9,6.5
2,LT,2018,1,1,2,14.2,0.26,0.7,1.7,50.0,46.0,27.0,0.0,74.0,1.3,67.0,67.0,7.3,6.2
3,LT,2018,1,1,3,14.3,0.25,0.7,1.6,49.0,38.0,24.0,0.0,74.0,1.4,68.0,64.0,6.3,6.1
4,LT,2018,1,1,4,14.2,0.25,0.6,1.9,48.0,37.0,23.0,0.0,74.0,1.5,61.0,58.0,5.0,5.2


In [5]:
project_data.tail()

,station,Year,Month,Day,Time,AMB_TEMP,CO,NO,NOx,O3,PM10,PM2.5,RAINFALL,RH,SO2,WD_HR,WIND_DIREC,WIND_SPEED,WS_HR
8755,LT,2018,12,31,19,14.5,0.24,1.5,9.4,34.0,10.0,13.0,0.0,95.0,2.3,58.0,57.0,7.0,6.4
8756,LT,2018,12,31,20,14.5,0.24,1.4,9.2,34.0,11.0,17.0,0.0,95.0,2.2,53.0,50.0,5.6,5.9
8757,LT,2018,12,31,21,14.6,0.25,1.5,9.1,33.0,8.0,13.0,0.0,95.0,1.9,52.0,54.0,5.7,5.5
8758,LT,2018,12,31,22,14.6,0.22,1.2,7.7,34.0,6.0,4.0,0.0,95.0,2.1,49.0,48.0,6.1,5.6
8759,LT,2018,12,31,23,14.7,0.19,1.0,5.8,36.0,6.0,1.0,0.0,96.0,2.0,48.0,52.0,6.3,6.3


In [6]:
#Show columns name
list(project_data.columns.values)

['station',
 'Year',
 'Month',
 'Day',
 'Time',
 'AMB_TEMP',
 'CO',
 'NO',
 'NOx',
 'O3',
 'PM10',
 'PM2.5',
 'RAINFALL',
 'RH',
 'SO2',
 'WD_HR',
 'WIND_DIREC',
 'WIND_SPEED',
 'WS_HR']

In [7]:
#Convert WD_HR to 8 degree
project_data['WD_HR'] = np.where(project_data['WD_HR'].between(0,22.5),1, project_data['WD_HR'])
project_data['WD_HR'] = np.where(project_data['WD_HR'].between(22.5,67.5),2, project_data['WD_HR'])
project_data['WD_HR'] = np.where(project_data['WD_HR'].between(67.5,112.5),3, project_data['WD_HR'])
project_data['WD_HR'] = np.where(project_data['WD_HR'].between(112.5,157.5),4, project_data['WD_HR'])
project_data['WD_HR'] = np.where(project_data['WD_HR'].between(157.5,202.5),5, project_data['WD_HR'])
project_data['WD_HR'] = np.where(project_data['WD_HR'].between(202.5,247.5),6, project_data['WD_HR'])
project_data['WD_HR'] = np.where(project_data['WD_HR'].between(247.5,292.5),7, project_data['WD_HR'])
project_data['WD_HR'] = np.where(project_data['WD_HR'].between(292.5,337.5),8, project_data['WD_HR'])
project_data['WD_HR'] = project_data['WD_HR'].mask(project_data['WD_HR'] > 337.5, 1)

In [8]:
#Convert WIND_DIREC to 8 degree
project_data['WIND_DIREC'] = np.where(project_data['WIND_DIREC'].between(0,22.5),1, project_data['WIND_DIREC'])
project_data['WIND_DIREC'] = np.where(project_data['WIND_DIREC'].between(22.5,67.5),2, project_data['WIND_DIREC'])
project_data['WIND_DIREC'] = np.where(project_data['WIND_DIREC'].between(67.5,112.5),3, project_data['WIND_DIREC'])
project_data['WIND_DIREC'] = np.where(project_data['WIND_DIREC'].between(112.5,157.5),4, project_data['WIND_DIREC'])
project_data['WIND_DIREC'] = np.where(project_data['WIND_DIREC'].between(157.5,202.5),5, project_data['WIND_DIREC'])
project_data['WIND_DIREC'] = np.where(project_data['WIND_DIREC'].between(202.5,247.5),6, project_data['WIND_DIREC'])
project_data['WIND_DIREC'] = np.where(project_data['WIND_DIREC'].between(247.5,292.5),7, project_data['WIND_DIREC'])
project_data['WIND_DIREC'] = np.where(project_data['WIND_DIREC'].between(292.5,337.5),8, project_data['WIND_DIREC'])
project_data['WIND_DIREC'] = project_data['WIND_DIREC'].mask(project_data['WIND_DIREC'] > 337.5, 1)

In [9]:
#Convert WIND_SPEED to 4 degree
project_data['WIND_SPEED'] = np.where(project_data['WIND_SPEED'].between(0,0.3),1, project_data['WIND_SPEED'])
project_data['WIND_SPEED'] = np.where(project_data['WIND_SPEED'].between(0.3,3.4),2, project_data['WIND_SPEED'])
project_data['WIND_SPEED'] = np.where(project_data['WIND_SPEED'].between(3.4,8),3, project_data['WIND_SPEED'])
project_data['WIND_SPEED'] = project_data['WIND_SPEED'].mask(project_data['WIND_SPEED'] > 8, 4)

In [10]:
#Convert WS_HR to 4 degree
project_data['WS_HR'] = np.where(project_data['WS_HR'].between(0,0.3),1, project_data['WS_HR'])
project_data['WS_HR'] = np.where(project_data['WS_HR'].between(0.3,3.4),2, project_data['WS_HR'])
project_data['WS_HR'] = np.where(project_data['WS_HR'].between(3.4,8),3, project_data['WS_HR'])
project_data['WS_HR'] = project_data['WS_HR'].mask(project_data['WS_HR'] > 8, 4)

In [11]:
#Convert type to integer because of one hot encoding 
project_data['WS_HR']= project_data['WS_HR'].round(0).astype(int)
project_data['WD_HR']= project_data['WD_HR'].round(0).astype(int)
project_data['WIND_DIREC']= project_data['WIND_DIREC'].round(0).astype(int)
project_data['WIND_SPEED']= project_data['WIND_SPEED'].round(0).astype(int)

In [12]:
#One hot encoding
WD_HR_OHE = pd.get_dummies(project_data['WD_HR'],prefix=['WD_HR_OHE'])
WIND_DIREC_OHE = pd.get_dummies(project_data['WIND_DIREC'],prefix=['WIND_DIREC_OHE'])
WIND_SPEED_OHE = pd.get_dummies(project_data['WIND_SPEED'],prefix=['WIND_SPEED_OHE'])
WS_HR_OHE = pd.get_dummies(project_data['WS_HR'],prefix=['WS_HR_OHE'])

In [13]:
#Concatenate dataframe
project_data_OHE = pd.concat([project_data, WD_HR_OHE,WIND_DIREC_OHE,WIND_SPEED_OHE,WS_HR_OHE], axis=1)
project_data_OHE.head()

,station,Year,Month,Day,Time,AMB_TEMP,CO,NO,NOx,O3,...,['WIND_DIREC_OHE']_5,['WIND_DIREC_OHE']_6,['WIND_DIREC_OHE']_7,['WIND_DIREC_OHE']_8,['WIND_SPEED_OHE']_2,['WIND_SPEED_OHE']_3,['WIND_SPEED_OHE']_4,['WS_HR_OHE']_2,['WS_HR_OHE']_3,['WS_HR_OHE']_4
0,LT,2018,1,1,0,14.0,0.25,0.4,2.4,46.0,...,0,0,0,0,0,1,0,0,1,0
1,LT,2018,1,1,1,14.1,0.25,0.8,1.7,49.0,...,0,0,0,0,0,1,0,0,1,0
2,LT,2018,1,1,2,14.2,0.26,0.7,1.7,50.0,...,0,0,0,0,0,1,0,0,1,0
3,LT,2018,1,1,3,14.3,0.25,0.7,1.6,49.0,...,0,0,0,0,0,1,0,0,1,0
4,LT,2018,1,1,4,14.2,0.25,0.6,1.9,48.0,...,0,0,0,0,0,1,0,0,1,0


In [14]:
#Select columns by droping columns
project_data_OHE.drop(project_data_OHE.columns[[0,1,2,3,4,15,16,17,18]], axis = 1, inplace = True)
project_data_OHE.head()

,AMB_TEMP,CO,NO,NOx,O3,PM10,PM2.5,RAINFALL,RH,SO2,...,['WIND_DIREC_OHE']_5,['WIND_DIREC_OHE']_6,['WIND_DIREC_OHE']_7,['WIND_DIREC_OHE']_8,['WIND_SPEED_OHE']_2,['WIND_SPEED_OHE']_3,['WIND_SPEED_OHE']_4,['WS_HR_OHE']_2,['WS_HR_OHE']_3,['WS_HR_OHE']_4
0,14.0,0.25,0.4,2.4,46.0,58.0,19.0,0.0,75.0,1.0,...,0,0,0,0,0,1,0,0,1,0
1,14.1,0.25,0.8,1.7,49.0,58.0,23.0,0.0,75.0,1.4,...,0,0,0,0,0,1,0,0,1,0
2,14.2,0.26,0.7,1.7,50.0,46.0,27.0,0.0,74.0,1.3,...,0,0,0,0,0,1,0,0,1,0
3,14.3,0.25,0.7,1.6,49.0,38.0,24.0,0.0,74.0,1.4,...,0,0,0,0,0,1,0,0,1,0
4,14.2,0.25,0.6,1.9,48.0,37.0,23.0,0.0,74.0,1.5,...,0,0,0,0,0,1,0,0,1,0


In [15]:
#One hour
y = project_data_OHE.iloc[1:,6]
y.head()

1    23.0
2    27.0
3    24.0
4    23.0
5    21.0
Name: PM2.5, dtype: float64

In [16]:
#Fetch x which needed to normalizate
x = project_data_OHE.iloc[:,0:10]
x.head(10)

,AMB_TEMP,CO,NO,NOx,O3,PM10,PM2.5,RAINFALL,RH,SO2
0,14.0,0.25,0.4,2.4,46.0,58.0,19.0,0.0,75.0,1.0
1,14.1,0.25,0.8,1.7,49.0,58.0,23.0,0.0,75.0,1.4
2,14.2,0.26,0.7,1.7,50.0,46.0,27.0,0.0,74.0,1.3
3,14.3,0.25,0.7,1.6,49.0,38.0,24.0,0.0,74.0,1.4
4,14.2,0.25,0.6,1.9,48.0,37.0,23.0,0.0,74.0,1.5
5,14.2,0.26,0.8,2.7,48.0,42.0,21.0,0.0,74.0,1.5
6,14.4,0.30,0.7,3.2,47.0,43.0,19.0,0.0,75.0,1.4
7,14.7,0.33,1.0,4.7,45.0,40.0,20.0,0.0,73.0,1.6
8,15.7,0.35,1.6,6.6,43.0,44.0,16.0,0.0,70.0,1.8
9,16.8,0.37,1.8,6.9,44.0,38.0,19.0,0.0,68.0,1.7


In [17]:
#Min-Max Normalization
x=np.array(x).astype(float)
x=(x-x.min(axis=0))/(x.max(axis=0)-x.min(axis=0))
x

array([[0.24837939, 0.27721163, 0.04343969, ..., 0.        , 0.60416164,
        0.18010567],
       [0.25126752, 0.27721163, 0.08687939, ..., 0.        , 0.60416164,
        0.25214794],
       [0.25415566, 0.29180172, 0.07601947, ..., 0.        , 0.5883281 ,
        0.23413737],
       ...,
       [0.26570819, 0.27721163, 0.16289886, ..., 0.        , 0.92083233,
        0.34220077],
       [0.26570819, 0.23344137, 0.13031908, ..., 0.        , 0.92083233,
        0.3782219 ],
       [0.26859632, 0.18967111, 0.10859924, ..., 0.        , 0.93666586,
        0.36021134]])

In [18]:
#Select aftering one hot encoding data
x_OHE = project_data_OHE.iloc[:,10:]
x_OHE.head()

,['WD_HR_OHE']_1,['WD_HR_OHE']_2,['WD_HR_OHE']_3,['WD_HR_OHE']_4,['WD_HR_OHE']_5,['WD_HR_OHE']_6,['WD_HR_OHE']_7,['WD_HR_OHE']_8,['WIND_DIREC_OHE']_1,['WIND_DIREC_OHE']_2,...,['WIND_DIREC_OHE']_5,['WIND_DIREC_OHE']_6,['WIND_DIREC_OHE']_7,['WIND_DIREC_OHE']_8,['WIND_SPEED_OHE']_2,['WIND_SPEED_OHE']_3,['WIND_SPEED_OHE']_4,['WS_HR_OHE']_2,['WS_HR_OHE']_3,['WS_HR_OHE']_4
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
2,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,1,0
3,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,1,0
4,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,1,0


In [19]:
#Convert dataframe to array
x_OHE.values

array([[0, 0, 1, ..., 0, 1, 0],
       [0, 0, 1, ..., 0, 1, 0],
       [0, 1, 0, ..., 0, 1, 0],
       ...,
       [0, 1, 0, ..., 0, 1, 0],
       [0, 1, 0, ..., 0, 1, 0],
       [0, 1, 0, ..., 0, 1, 0]], dtype=uint8)

In [20]:
x_final = np.concatenate((x, x_OHE), axis=1)
x_final

array([[0.24837939, 0.27721163, 0.04343969, ..., 0.        , 1.        ,
        0.        ],
       [0.25126752, 0.27721163, 0.08687939, ..., 0.        , 1.        ,
        0.        ],
       [0.25415566, 0.29180172, 0.07601947, ..., 0.        , 1.        ,
        0.        ],
       ...,
       [0.26570819, 0.27721163, 0.16289886, ..., 0.        , 1.        ,
        0.        ],
       [0.26570819, 0.23344137, 0.13031908, ..., 0.        , 1.        ,
        0.        ],
       [0.26859632, 0.18967111, 0.10859924, ..., 0.        , 1.        ,
        0.        ]])

In [21]:
def creat_dataset(ds, look_back=1):
    x_data = []
    for i in range(len(ds[:-1])-(look_back-1)):
        tmp = []
        for j in range(look_back):
            tmp+=list(ds[i+j])
        x_data.append(tmp)        
    return np.array(x_data)

In [22]:
#Call function
look_back = 1
x_train = creat_dataset(x_final, look_back)
print('Looking back hour:', look_back)
print('x_train shape:', x_train.shape)
# print('y_train shape:', y_train.shape)

Looking back hour: 1
x_train shape: (8759, 32)


In [23]:
int(8758*0.7)

6130

In [24]:
#7:3
X_train, Y_train = x_train[:6130], y[:6130]
X_test, Y_test = x_train[6130:], y[6130:]

In [25]:
#Convert dict to array
Y_train.values

array([23., 27., 24., ..., 19., 21., 24.])

In [26]:
#SVR
clf = SVR(kernel='rbf', C=1e3, gamma=0.1)

In [27]:
#Insert train data
clf.fit(X_train, Y_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [28]:
#Insert test data into model
predict_y = clf.predict(X_test)

In [29]:
#Adjust predict_y structure
predict_y = np.reshape(predict_y, (-1, 1))

In [30]:
#Adjust Y_test structure
Y_test = Y_test.values.reshape(-1,1)

In [31]:
#Show accuracy
clf.score(X_test,Y_test)

0.7990535107910729

In [32]:
#Save model
joblib.dump(clf,"./LT_ML_Predict.pkl")

['./LT_ML_Predict.pkl']